In [22]:
!pip -q install rank_bm25
!pip -q install -U sentence-transformers
!pip -q install deep_translator
!pip -q install transformers==4.34.0 datasets==2.14.5 accelerate==0.23.0 evaluate==0.4.1 peft==0.5.0

In [6]:
#Inference
import torch
import numpy as np
import pandas as pd
from context_retrieval.context_retrieval_vimedqa import *

import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [9]:
#Model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
model.load_state_dict(torch.load("./model__generator_FLAN-T5/flan-t5-base/checkpoint-22000/pytorch_model.bin",
                                 map_location=torch.device('cpu')))

<All keys matched successfully>

In [20]:
def predict_per_sample(df, model, tokenizer):
  convert_to_binary = {'a': "10000", 'b':"01000", 'c':"00100", 'd':"00010", 'e':"00001",
                       'ab': "11000", 'ac':"10100", 'ad':"10010", 'ae':"10001",
                       'bc':"01100", 'bd':"01010", 'be':"01001", 'cd':"00110", 'ce':"00101", 'de':"00011",
                       'abc': "11100", 'abd':"11010", 'abe':"11001", 'acd':"10110", 'ace':"10101", 'ade':"10011",
                       'bcd': "01110", 'bce':"01101", 'bde':"01011", 'cde':"00111",
                       'abcd':"11110", 'abce':"11101", 'abde':"11011", 'acde':"10111", 'bcde':"01111",'abcde':"11111"}
  answers = []
  for idx, row in df.iterrows():
    context = row["context"]
    question = row["question"]
    choices = [op for op in row[["option_1", "option_2", "option_3", "option_4", "option_5", "option_6"]].tolist() if str(op) != '']
    text_choices = ". \n ".join(choices)
    prompt = f"Question: {question}. Choice the correct answers from: {text_choices}. Context: {context}."
    model_inputs = tokenizer(prompt, max_length=512, truncation=True, return_tensors="pt")
    outputs = model.generate(**model_inputs,  max_new_tokens=20)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if prediction in convert_to_binary:
      answer = convert_to_binary[prediction]
    else:
      answer = '00000'
    answer = answer[:len(choices)]
    answers.append(answer)
  return answers

def prediction(test_df_path, model, tokenizer):
  test_data = test_set_preprocessing(test_df_path).iloc[:6,:]
  corpus_path = './modified_dataset/ViMed_corpus_dict.csv'
  corpus_df = corpus_preprocessing(corpus_path)
  test_w_context_data = bm25_searcher(test_data, corpus_df, top_k=5)
  #test_w_context_data.to_csv("vi_context_public_test.csv", index = False)
  en_test_w_context_data = translate_df(test_w_context_data)
  #en_test_w_context_data.to_csv("en_context_public_test.csv")
  return predict_per_sample(en_test_w_context_data, model, tokenizer)

In [21]:
test_df_path = "./modified_dataset/public_test.csv"
answers = prediction(test_df_path, model, tokenizer)
answers

6it [02:33, 25.63s/it]


['0100', '0100', '0100', '01', '0100', '0010']